In [1]:
import os
import zipfile
import sys
import mysql.connector
import requests
import re


sys.path.append('/home/lib')
nodeServer = os.environ['NODE_SERVER']
db_host = os.environ['SQL_HOST']
db_user = os.environ['SQL_USER']
db_password = os.environ['SQL_PASSWORD']
database = os.environ['DATABASE']

# MySQL 서버에 연결
try:
    conn = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=database
    )
    print("MySQL에 정상적으로 연결되었습니다.")
    cur = conn.cursor()

except mysql.connector.Error as e:
    print(f"MySQL 연결 오류: {e}")

finally:
    
    # 압축 파일 경로
    zip_path = "./메인이미지완료/main_finish.zip"
    
    # 압축 파일 열기
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # 압축 해제
        zip_ref.extractall('./메인이미지완료')
        print("압축 해제 완료")
        
    
    root_folder = './메인이미지완료/main_finish'
    
    # root_folder 내의 파일 목록을 가져오기
    files_in_folder = [f for f in os.listdir(root_folder) if os.path.isfile(os.path.join(root_folder, f)) and not f.startswith('.')]
    
    # 각 파일에 대한 작업 수행
    for file_name in files_in_folder:
        try:
            file_path = os.path.join(root_folder, file_name)
            
            # 파일을 열어서 requests 모듈에서 사용할 수 있는 형태로 변환
            files = {'file': (file_name, open(file_path, 'rb'))}
            
            refUniqueId = re.search(r'\d+', file_name).group()
            
            updateStylingSuggestionProduct = f"UPDATE image_file SET delete_yn = 'Y' WHERE ref_unique_id = {refUniqueId} AND image_file_group_id =6"
            
            cur.execute(updateStylingSuggestionProduct)
            
            conn.commit()
      
            data = {'imageFileGroupCode':'styling-suggestion-main', 'refUniqueId':refUniqueId}    
            
            response = requests.post(f"{nodeServer}common/v1/image", data=data, files=files).json()
            print(response)
            if response['code'] == '0':
                print(f'{refUniqueId} 이미지 업로드 성공')
            else :
                print(f'{refUniqueId} 이미지 업로드 실패')
                    
        except e:
            print(e)
            print(f'{refUniqueId} 이미지 업로드 실패')

    conn.close()



MySQL에 정상적으로 연결되었습니다.
압축 해제 완료
{'code': '9000', 'createdAt': '2023-11-27 22:12:41', 'message': ['property refUniqueId should not exist']}
5730 이미지 업로드 실패
